# Simple Rating Biases
* Computes a bias for each user and for each item
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = m + u_i + a_j$
* $m = \text{mean}_{ij}(r_{ij})$
* $u_i = \text{mean}_j(r_{ij}) - m$
* $a_j = \text{mean}_i(r_{ij}) - m$

In [ ]:
medium = "";

In [ ]:
using DataFrames
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../Alpha.ipynb");

In [ ]:
const metric = "rating"
const name = "$medium/$metric/SimpleBiases"
set_logging_outdir(name);

## Training

In [ ]:
function get_df()
    df = get_split(
        "training",
        metric,
        medium,
        [:userid, :itemid, :rating],
        "$medium/$metric/Simple",
    )
    DataFrame(user = df.userid, item = df.itemid, rating = df.rating)
end;

In [ ]:
const training = get_df();

In [ ]:
const μ = mean(training.rating);

In [ ]:
u = combine(groupby(training, :user), :rating => mean => :rating)
u = Dict(Pair.(u.user, u.rating .- μ));

In [ ]:
a = combine(groupby(training, :item), :rating => mean => :rating)
a = Dict(Pair.(a.item, a.rating .- μ));

## Inference

In [ ]:
function make_prediction(users, items, u, a, μ)
    r = zeros(Float32, length(users))
    @showprogress for i = 1:length(r)
        rating = μ
        if users[i] in keys(u)
            rating += u[users[i]]
        end
        if items[i] in keys(a)
            rating += a[items[i]]
        end
        r[i] = rating
    end
    r
end;

model(users, items) = make_prediction(users, items, u, a, μ);

In [ ]:
write_params(Dict("u" => u, "a" => a, "μ" => μ), name);

In [ ]:
write_alpha(model, medium, name, ["training", "test", "negative"]);

In [ ]:
print_losses([name, "$medium/$metric/Simple"], metric, medium, ["training", "test"]);